In [1]:
#   Se importan los paquetes basicos para encontrar el perfil de linea

from DefSD import *
#from astropy.analytic_functions import blackbody
#from astropy.convolution import convolve, Gaussian1DKernel, Box1DKernel
import os


In [50]:
###    Se verifica que existan todos los archivos que se van a ajustar.
#      Se puede omitir para trabajar un sol archivo

fileName = 'list3'
dirName = 'LIST3'
if not os.path.exists(dirName):
    print 'el directorio', dirName, 'no existe.'
else:
    resultListDT=Table.read(dirName+'/'+fileName+'.tbl', format='ascii')       #testSDSSFound.txt
    for x in range(len(resultListDT['ObjID'])):
        name = resultListDT['ObjID'][x]
        try:
            hdulist = fits.open(dirName+'/'+name+'.fits')
        except IOError:
            print "Error opening the file: ", name

In [51]:
#   a table resultListDT relates the name of the objects and the resulting temperature and density
#   for the four couples of species

#resultListDT.tbl

resultListDT=Table.read(dirName+'/'+fileName+'.tbl', format='ascii')       #testSDSSFound.txt
ion = [' [OIII]', ' [SII]a', ' [NII]', ' [OII]a', ' [NII]', ' [SII]a', ' [OIII]', ' [OII]a',' [OIII]', ' [ArIV]', ' [NII]', ' [ArIV]', ' [OII]b', ' [SII]b',' [OII]b', ' [ArIV]',' [SII]b',' [ArIV]', ' [SII]b',' [OII]a', ' [OIII]', ' [SII]c', ' [NII]', ' [SII]c', ' [NeIII]', ' [OII]a', ' [NeIII]', ' [SII]a', ' [NeIII]', ' [ArIV]', ' [NeIII]', ' [SII]c']#                  
for i in range(1,17):
    colT= Column(name='Temp_'+str(i)+ion[2*(i-1)], length=len(resultListDT),unit=u.K)
    colD= Column(name='Den_'+str(i)+ion[2*(i-1)+1], length=len(resultListDT),unit=1 / (u.cm**3))
    resultListDT.add_columns([colT,colD])
resultListDT

#   Obtain the lists Spectra, Headers, spectrum variance of the intensity according with a list of galaxy names
#   and a Table Tabla of models for each line for all the spectra

Headers = []
Spectra = []
Spectra2 = []
SpectraCont = []
Lambda = []
Modelos=[]
Tablas=[]
HDUs=[]
TypeFITS=[]
SpectraIvar = []
LINEProfile = []
paramCont=0.4
typeFITS =0
for x in range(len(resultListDT['ObjID'])):
    try:
        name = resultListDT['ObjID'][x]
        z=resultListDT['z'][x]
        isFits = resultListDT['isFits'][x]
        if isFits:
            hdulist = fits.open(dirName+'/'+name+'.fits')
            head= hdulist[0].header
            typeFITS=0
            if head['NAXIS'] == 0:
                total= hdulist[1].data
            elif head['NAXIS'] == 1:
                typeFITS=1
            if typeFITS==0:
                spectraFlux = total['flux']
                spectraIvar = total['ivar']
                linlam=np.exp(total['loglam']*np.log(10))          #  ocultar para leer desde archivo
                lambd = linlam/(1+z)
            else:
                spectraFlux = hdulist[0].data #*(1e-19)
                spectraIvar = spectraFlux*0
                if 'CDELT1' in head:
                    lambd = np.linspace(head['CRVAL1'], head['CRVAL1']+head['CDELT1']*(len(spectraFlux)-1), len(spectraFlux))
                elif 'W0' in head:
                    lambd = np.linspace(head['W0'], head['W0']+head['WPC']*(len(spectraFlux)-1), len(spectraFlux))
                else:
                    print "Can't identify FITS format in file: ", str(name)
                    break
                lambd = lambd/(1+z)
        else:
            specTable = Table.read(dirName+'/'+str(name)+'.txt', format='ascii')
            spectraFlux = specTable['Intensity'].data #.tolist()
            spectraIvar = spectraFlux*0
            lambd = specTable['Spectral-axis'].data #.tolist()
            lambd = lambd/(1+z)
        orderU = 10**(int(mt.log(np.mean(spectraFlux), 10))-1)

        TypeFITS.append(typeFITS)
        Spectra2.append(spectraFlux)
        print "Object [",x,"]: ", name
        spec =np.array(spectraFlux)
        spec2, lambd = r(spec, lambd, paramCont*orderU)
        
        #gauss_kernel = Gaussian1DKernel(2)
        #smoothed_data_gauss = convolve(spectraFlux, gauss_kernel)
        #spec2 = smoothed_data_gauss

        limResolucion = mt.ceil(lambd[1]-lambd[0])

    # ..............    

        # Este ciclo calcula remplaza la región del contínuo que está justo en el perfil de línea, por una recta
        # que es mas preciso al determinar la amplitud del modelo gaussiano
        ListaLineas=Table.read('LinesAll.txt', format='ascii')
        lineProf = lineProfile(x,spec2, lambd,ListaLineas,Spectra2, limResolucion)
        LINEProfile.append(lineProf)
        for ina in range(len(lineProf)):
            lInf = lineProf[ina]['inf']
            lSup = lineProf[ina]['sup']
            lDel = lSup-lInf
            if (lInf-3*lDel) >= 0 :
                lInf = lInf-3*lDel
            if (lSup+3*lDel) < len(lambd) :
                lSup = lSup+3*lDel
            m1, b1 = linModel( lambd[lInf] , spec2[lInf] , lambd[lSup], spec2[lSup])
            lineLambd = lambd[lInf:lSup]
            lineIntCont = lines( lineLambd , m1, b1)
            spec2[lInf:lSup] = lineIntCont
        LS=np.zeros([len(ListaLineas),11])
        ModelG1, lineProf, spec2 = finalFit(x, spec2, lambd,ListaLineas,Spectra2, spectraIvar,2000, limResolucion)
#        ModelL1 = fit2(x, spec2, lambd,ListaLineas,Spectra,2500)

        Tabla=Table(LS,names=('Amplitude','Wavelength','Sigma','Lambda Vac','Element','Flux','err', 'FWHM', 'eqWidth', 'S/N>3', 'LINE'),
                    dtype=('f8','f8','f8','f8','S8','f8','f8','f8','f8','S5', 'S12'))
        Tabla['Lambda Vac']=ListaLineas['LAMBDA VAC ANG']
        Tabla['Element']=ListaLineas['SPECTRUM']
        AA=split_list(ModelG1[0].parameters, wanted_parts=len(ListaLineas))
        AAA=split_list(ModelG1[1].parameters, wanted_parts=2)
    #    AA=split_list(ModelL1[0].parameters, wanted_parts=len(ListaLineas))
    #    AAA=split_list(ModelL1[1].parameters, wanted_parts=2)
        Tabla['LINE']=ListaLineas['LINE']
        for ii in range(len(ListaLineas)):
            if 3*np.mean(spectraIvar[lineProf[0]['inf']:lineProf[0]['sup']]) >= AA[ii][0]:
                Tabla['S/N>3'][ii] = str(False)
            else:
                Tabla['S/N>3'][ii] = str(True)
            if AA[ii][2]<0:
                Tabla['Amplitude'][ii],Tabla['Wavelength'][ii],Tabla['Sigma'][ii]=np.array([AA[ii][0], AA[ii][1], mt.fabs(AA[ii][2])])
            else:
                Tabla['Amplitude'][ii],Tabla['Wavelength'][ii],Tabla['Sigma'][ii]=np.array([AA[ii][0], AA[ii][1], AA[ii][2]])
            if ii==0 or ii==1:
                if AAA[ii][2]<0:
                    Tabla['Amplitude'][ii],Tabla['Wavelength'][ii],Tabla['Sigma'][ii]=np.array([AAA[ii][0], AAA[ii][1], mt.fabs(AAA[ii][2])])
                else:
                    Tabla['Amplitude'][ii],Tabla['Wavelength'][ii],Tabla['Sigma'][ii]=np.array([AAA[ii][0], AAA[ii][1], AAA[ii][2]])
        Modelos.append(ModelG1)
        #    Modelos.append(ModelL1)
        #print Tabla 
        Tablas.append(Tabla)
        SpectraIvar.append(spectraIvar)
        SpectraCont.append(spec2)
        Lambda.append(lambd)
        Spectra.append(spectraFlux)
    except ValueError:
        print "It has not been possible to process the spectrum of: ", name

Object [ 0 ]:  IRAS15225+2350SP1
It has not been possible to fit the line:  6529.04
It has not been possible to fit the line:  5578.8874
It has not been possible to fit the line:  3971.195


In [19]:
#   plot of the galaxy with index jj observed with respect to the continuum in zero. Two models for the lines required 
%matplotlib
### inline
for jj in [0]:  #  range(len(resultListDT)):
#    plt.errorbar(Lambda[jj],Spectra[jj],SpectraIvar[jj],  c='gray',lw=1,label='Spectrum')
    plt.plot(Lambda[jj],Spectra[jj], lw=0.5, c = 'k', label='Spectra')
    plt.plot(Lambda[jj],SpectraCont[jj], lw=2, c = 'g', label='Spectra')
    lPPP = lineProf
    for iasiao in range(len(lPPP)):
        plt.plot(Lambda[jj][lPPP['inf'][iasiao]:lPPP['sup'][iasiao]],Spectra[jj][lPPP['inf'][iasiao]:lPPP['sup'][iasiao]], lw=2.0, c = 'b')


Using matplotlib backend: Qt4Agg


###     EqWidth ,  FWHM,  FLUX

In [52]:
#   The plot of relative intensity and continuum emission for the galaxy index jj and the wavelenght of all the lines
#   The equivalent width, flux and FWHM for galaxy jj is measured too   

if not os.path.exists(dirName+'/resultTables'):
    os.makedirs(dirName+'/resultTables')

for jj in range(len(Tablas)):
    spe = []
    speCont = []
    for xind in range(len(Spectra[jj])):
        spe.append((Spectra[jj][xind])/(SpectraCont[jj][xind]))
        speCont.append((SpectraCont[jj][xind])/(SpectraCont[jj][xind]))

    lFWHM = []
    eqW = []    ##  lista de valores 
    #  una vez normalizado el espectro observado a la medida del contínuo se integra por el método de Simpson
    for iLineProf in range(len(LINEProfile[jj])):
        try:
            f = FWHM(Lambda[jj][LINEProfile[jj]['inf'][iLineProf]:LINEProfile[jj]['sup'][iLineProf]], Spectra[jj][LINEProfile[jj]['inf'][iLineProf]:LINEProfile[jj]['sup'][iLineProf]])
            if len(f) == 0:
                lFWHM.append(0)
            else:
                lFWHM.append(f[0])
        except ValueError:
            lFWHM.append(0)
        if (LINEProfile[jj]['inf'][iLineProf] == LINEProfile[jj]['sup'][iLineProf]) and (LINEProfile[jj]['sup'][iLineProf] == 0):
            eqW.append(0)
        else:
            sInteg = simps(y=spe[LINEProfile[jj]['inf'][iLineProf]:LINEProfile[jj]['sup'][iLineProf]], x=Lambda[jj][LINEProfile[jj]['inf'][iLineProf]:LINEProfile[jj]['sup'][iLineProf]], even='avg')
            eqW.append(sInteg)
    #   Flux determination for each model in galaxy list
    #   assignment of the FWHM and equivalent width in the Table for the first galaxy
    for i in range(len(Tablas[jj])):
        if Tablas[jj][i]['Amplitude']<=0:
            Tablas[jj]['Flux'][i],Tablas[jj]['err'][i]= 0,0
        else:
            Tablas[jj]['Flux'][i],Tablas[jj]['err'][i]=quad(gaussian,Tablas[jj][i]['Wavelength']-3*Tablas[jj][i]['Sigma'],Tablas[jj][i]['Wavelength']+3*Tablas[jj][i]['Sigma'],args=(Tablas[jj][i]['Amplitude'],Tablas[jj][i]['Wavelength'],Tablas[jj][i]['Sigma']))
            Tablas[jj]['FWHM'][i],Tablas[jj]['eqWidth'][i] = lFWHM[i], eqW[i]
    Tablas[jj].write(dirName+'/resultTables/Tabla'+str(resultListDT['ObjID'][jj])+'.tbl',format='ascii')
    
    TablePyNeb = Tablas[jj]['LINE', 'Flux', 'err']
    
    TablePyNeb.write(dirName+'/resultTables/TablaPN'+str(resultListDT['ObjID'][jj])+'.dat',format='ascii.tab')
#Tablas[jj]

###     Corrección por atenuación debida al polvo

In [53]:
###    Corrección por atenuación debida al polvo de acuerdo a la ley CCM89 con la razón entre
#      Ha y Hb tomando el valor esperado de 2.86 con R_v = 3.1
#      También se puede utilizar el cociente Hgamma / Hb 


#      Esta parte fue tomada del documento de PyNeb para diagramas diagnósticos de las
#      cinco líneas prohibidas seleccionadas


### General settings
# Setting verbosity level. Enter pn.my_logging? for details
pn.log_.level = 2 # set this to 3 to have more details
# Adopt an extinction law
extinction_law = 'CCM89'

for jj in range(len(Tablas)):
    
    # Define the data file
    obs_data = dirName+'/resultTables/TablaPN'+str(resultListDT['ObjID'][jj])+'.dat'
    # Define plot title
    title = 'Flux'
    ### Read and deredden observational data
    # define an Observation object and assign it to name ’obs’
    obs = pn.Observation()

    # fill obs with data read from file obs_data, with lines varying across rows and a default per
    obs.readData(obs_data, fileFormat='lines_in_rows_err_cols', errIsRelative=True)
    # deredden data with Cardelli’s law
    obs.extinction.law = extinction_law
    obs.corrected=True
    
    ###  Corrección del cociente Ha / Hb cuyo valor esperado es 2.86 para nebulosas planetarias
    obs.def_EBV(label1="H1r_6563A", label2="H1r_4861A", r_theo=2.86)
    obs.correctData(normWave=4861.)

    ###  Corrección del cociente Hgamma / Hb cuyo valor esperado es 0.47 para nebulosas planetarias
    #obs.def_EBV(label1="H1r_4341A", label2="H1r_4861A", r_theo=0.47)
    #obs.correctData(normWave=4861.)

    obs.correctData()
    ### Include the diagnostics of interest
    # instantiate the Diagnostics class
    diags = pn.Diagnostics()
    # include in diags the relevant line ratios
    diags.addDiag([
    '[NII] 5755/6584',
    '[OII] 3726/3729',
    '[OIII] 4363/5007',
    '[SII] 6731/6716',
    '[ArIV] 4740/4711',
    '[SII] 4069/4076',
    '[SII] 4072+/6720+',
    '[OII] 3727+/7325+',
    '[OI] 5577/6300',
    '[OI] 5577/6300+',
    '[NeIII] 3343/3930+',
    '[ArIII] 5192/7136',
    '[ArIII] 5192/7300+'  ])

    diags.addClabel('[SII] 6731/6716', '[SII]a')
    diags.addClabel('[OII] 3726/3729', '[OII]a')
    diags.addClabel('[OII] 3727+/7325+', '[OII]b')
    diags.addClabel('[SII] 4072+/6720+', '[SII]b')
    diags.addClabel('[SII] 4069/4076', '[SII]c')
    diags.addClabel('[OI] 5577/6300', '[OI]a')
    diags.addClabel('[OI] 5577/6300+', '[OI]b')
    diags.addClabel('[ArIII] 5192/7136', '[ArIII]a')
    diags.addClabel('[ArIII] 5192/7300+', '[ArIII]b')

    ext=obs.extinction
    print 'Extinction Law: CCM89'
    print 'Object['+str(jj)+'] : ', str(resultListDT['ObjID'][jj]), 'cHbeta =', ext.cHbeta, ' R_V =',  ext.R_V, ' E(B_V) =',  ext.E_BV
    
    TeN2O2, NeN2O2 = diags.getCrossTemDen('[NII] 5755/6584+', '[OII] 3726/3729', obs=obs)
    TeO3O2, NeO3O2 = diags.getCrossTemDen('[OIII] 4363/5007+', '[OII] 3726/3729', obs=obs)
    TeO3S2, NeO3S2 = diags.getCrossTemDen('[OIII] 4363/5007+', '[SII] 6731/6716', obs=obs)
    TeN2S2, NeN2S2 = diags.getCrossTemDen('[NII] 5755/6584+', '[SII] 6731/6716', obs=obs)
    TeO3Ar4, NeO3Ar4 = diags.getCrossTemDen('[OIII] 4363/5007+', '[ArIV] 4740/4711', obs=obs)
    TeN2Ar4, NeN2Ar4 = diags.getCrossTemDen('[NII] 5755/6584+', '[ArIV] 4740/4711', obs=obs)
    
    TeO2S2, NeO2S2 = (0,0)  #diags.getCrossTemDen('[OII] 3727+/7325+', '[SII] 6731/6716', obs=obs)
    TeO2Ar4, NeO2Ar4 = (0,0)  #diags.getCrossTemDen('[OII] 3727+/7325+', '[ArIV] 4740/4711', obs=obs)
    TeS2O2, NeS2O2 = diags.getCrossTemDen('[SII] 4072+/6720+', '[OII] 3726/3729', obs=obs)
    TeS2Ar4, NeS2Ar4 = diags.getCrossTemDen('[SII] 4072+/6720+', '[ArIV] 4740/4711', obs=obs)
    TeO3S2c, NeO3S2c = diags.getCrossTemDen('[OIII] 4363/5007+', '[SII] 4069/4076', obs=obs)
    TeN2S2c, NeN2S2c = diags.getCrossTemDen('[NII] 5755/6584+', '[SII] 4069/4076', obs=obs)

    TeNe3O2, NeNe3O2 = diags.getCrossTemDen('[NeIII] 3343/3930+', '[OII] 3726/3729', obs=obs)
    TeNe3S2, NeNe3S2 = diags.getCrossTemDen('[NeIII] 3343/3930+', '[SII] 6731/6716', obs=obs)
    TeNe3Ar4, NeNe3Ar4 = diags.getCrossTemDen('[NeIII] 3343/3930+', '[ArIV] 4740/4711', obs=obs)
    TeNe3S2c, NeNe3S2c = diags.getCrossTemDen('[NeIII] 3343/3930+', '[SII] 4069/4076', obs=obs)

    resultListDT['Temp_1 [OIII]'][jj], resultListDT['Den_1 [SII]a'][jj] = TeO3S2, NeO3S2
    resultListDT['Temp_3 [NII]'][jj], resultListDT['Den_3 [SII]a'][jj] = TeN2S2, NeN2S2
    resultListDT['Temp_2 [NII]'][jj], resultListDT['Den_2 [OII]a'][jj] = TeN2O2, NeN2O2
    resultListDT['Temp_4 [OIII]'][jj], resultListDT['Den_4 [OII]a'][jj] = TeO3O2, NeO3O2
    resultListDT['Temp_6 [NII]'][jj], resultListDT['Den_6 [ArIV]'][jj] = TeN2Ar4, NeN2Ar4
    resultListDT['Temp_5 [OIII]'][jj], resultListDT['Den_5 [ArIV]'][jj] = TeO3Ar4, NeO3Ar4
    
    resultListDT['Temp_7 [OII]b'][jj], resultListDT['Den_7 [SII]b'][jj] = TeO2S2, NeO2S2
    resultListDT['Temp_8 [OII]b'][jj], resultListDT['Den_8 [ArIV]'][jj] = TeO2Ar4, NeO2Ar4
    resultListDT['Temp_9 [SII]b'][jj], resultListDT['Den_9 [ArIV]'][jj] = TeS2Ar4, NeS2Ar4
    resultListDT['Temp_10 [SII]b'][jj], resultListDT['Den_10 [OII]a'][jj] = TeS2O2, NeS2O2
    resultListDT['Temp_11 [OIII]'][jj], resultListDT['Den_11 [SII]c'][jj] = TeO3S2c, NeO3S2c
    resultListDT['Temp_12 [NII]'][jj], resultListDT['Den_12 [SII]c'][jj] = TeN2S2c, NeN2S2c

    resultListDT['Temp_13 [NeIII]'][jj], resultListDT['Den_13 [OII]a'][jj] = TeNe3O2, NeNe3O2
    resultListDT['Temp_14 [NeIII]'][jj], resultListDT['Den_14 [SII]a'][jj] = TeNe3S2, NeNe3S2
    resultListDT['Temp_15 [NeIII]'][jj], resultListDT['Den_15 [ArIV]'][jj] = TeNe3Ar4, NeNe3Ar4
    resultListDT['Temp_16 [NeIII]'][jj], resultListDT['Den_16 [SII]c'][jj] = TeNe3S2c, NeNe3S2c

    #'[OII]b', '[SII]b','[OII]b', '[ArIV]','[SII]b','[ArIV]', '[SII]b','[OII]a'


Extinction Law: CCM89
Object[0] :  IRAS15225+2350SP1 cHbeta = [ 1.71540854]  R_V = 3.1  E(B_V) = [ 1.18831758]


###               Electron density and temperature results

In [54]:
#resultListDT=Table.read(dirName+'/resultTables/DeTe.tbl', format='ascii')
resultListDT

ObjID,ra,dec,z,isFits,Temp_1 [OIII],Den_1 [SII]a,Temp_2 [NII],Den_2 [OII]a,Temp_3 [NII],Den_3 [SII]a,Temp_4 [OIII],Den_4 [OII]a,Temp_5 [OIII],Den_5 [ArIV],Temp_6 [NII],Den_6 [ArIV],Temp_7 [OII]b,Den_7 [SII]b,Temp_8 [OII]b,Den_8 [ArIV],Temp_9 [SII]b,Den_9 [ArIV],Temp_10 [SII]b,Den_10 [OII]a,Temp_11 [OIII],Den_11 [SII]c,Temp_12 [NII],Den_12 [SII]c,Temp_13 [NeIII],Den_13 [OII]a,Temp_14 [NeIII],Den_14 [SII]a,Temp_15 [NeIII],Den_15 [ArIV],Temp_16 [NeIII],Den_16 [SII]c
,,,,,K,1 / cm3,K,1 / cm3,K,1 / cm3,K,1 / cm3,K,1 / cm3,K,1 / cm3,K,1 / cm3,K,1 / cm3,K,1 / cm3,K,1 / cm3,K,1 / cm3,K,1 / cm3,K,1 / cm3,K,1 / cm3,K,1 / cm3,K,1 / cm3
string136,float64,float64,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
IRAS15225+2350SP1,231.182922363,23.6697216034,0.1386644,1,67738.5521303,354.279494752,nan,nan,13952.7840205,212.460740696,nan,nan,35246.5164083,232814.335013,7207.01222134,142064.895701,0.0,0.0,0.0,0.0,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


In [55]:

resultListDT.write(dirName+'/resultTables/DeTe'+str(resultListDT['ObjID'][jj])+'.tbl',format='ascii')
resultListDT.write(dirName+'/resultTables/DeTe.tbl',format='ascii')

In [62]:
###    Grafica del diagnóstico densidad y temperatura para el espectro de indice jj

jj=0

%matplotlib

### General settings
# Setting verbosity level. Enter pn.my_logging? for details
pn.log_.level = 2 # set this to 3 to have more details
# Adopt an extinction law
extinction_law = 'CCM89'

# Define the data file
obs_data = dirName+'/resultTables/TablaPN'+str(resultListDT['ObjID'][jj])+'.dat'
# Define plot title
title = 'Flux'
### Read and deredden observational data
# define an Observation object and assign it to name ’obs’
obs = pn.Observation()

# fill obs with data read from file obs_data, with lines varying across rows and a default per
obs.readData(obs_data, fileFormat='lines_in_rows_err_cols', errIsRelative=True)
# deredden data with Cardelli’s law
obs.extinction.law = extinction_law

# de 1 a 7
corrHxHb = 7

if corrHxHb==1:
    ###  Corrección del cociente Ha / Hb cuyo valor esperado es 2.86 para nebulosas planetarias
    obs.def_EBV(label1="H1r_6563A", label2="H1r_4861A", r_theo=2.86)
    obs.correctData(normWave=4861.)
elif corrHxHb==2:
    ###  Corrección del cociente Hgamma / Hb cuyo valor esperado es 0.47 para nebulosas planetarias
    obs.def_EBV(label1="H1r_4341A", label2="H1r_4861A", r_theo=0.47)
    obs.correctData(normWave=4861.)
elif corrHxHb==3:
    ###  Corrección del cociente Hdelta / Hb cuyo valor esperado es 0.256 para nebulosas planetarias
    obs.def_EBV(label1="H1r_4102A", label2="H1r_4861A", r_theo=0.256)
    obs.correctData(normWave=4861.)
elif corrHxHb==4:
    ###  Corrección del cociente Hepsilon / Hb cuyo valor esperado es 0.158 para nebulosas planetarias
    obs.def_EBV(label1="H1r_3970A", label2="H1r_4861A", r_theo=0.158)
    obs.correctData(normWave=4861.)
elif corrHxHb==5:
    ###  Corrección del cociente H8 / Hb cuyo valor esperado es 0.105 para nebulosas planetarias
    obs.def_EBV(label1="H1r_3889A", label2="H1r_4861A", r_theo=0.105)
    obs.correctData(normWave=4861.)
elif corrHxHb==6:
    ###  Corrección del cociente H9 / Hb cuyo valor esperado es 0.0730 para nebulosas planetarias
    obs.def_EBV(label1="H1r_3835A", label2="H1r_4861A", r_theo=0.0730)
    obs.correctData(normWave=4861.)
elif corrHxHb==7:
    ###  Corrección del cociente H9 / Hb cuyo valor esperado es 0.0529 para nebulosas planetarias
    obs.def_EBV(label1="H1r_3798A", label2="H1r_4861A", r_theo=0.0529)
    obs.correctData(normWave=4861.)

obs.correctData()
obs.corrected=True

# abundancies taken selected Te and De for 3c234
#Te = np.mean([resultListDT['Temp_1 [OIII]'][0], resultListDT['Temp_4 [OIII]'][0] ])
#Ne = np.mean([resultListDT['Den_1 [SII]a'][0], resultListDT['Den_4 [OII]a'][0] ])

Te = np.mean([resultListDT['Temp_4 [OIII]'][0], resultListDT['Temp_5 [OIII]'][0] ])
Ne = np.mean([resultListDT['Den_4 [OII]a'][0], resultListDT['Den_10 [OII]a'][0] ])


# Define a dictionary to hold all the Atom objects needed
all_atoms = pn.getAtomDict(atom_list=obs.getUniqueAtoms())
# define a dictionary to store the abundances
ab_dict = {}
# we use the following lines to determine the ionic abundances
ab_labels = ['O2_3726A', 'O2_3729A','O3_4363A', 'O3_5007A', 'O3_4931A','O3_4959A', 'N2_5755A','N2_6527A', 'S2_6716A','N2_6548A'
             ,'N2_6584A', 'S2_6731A','O2_7319A+', 'O2_7330A+', 'S2_4069A', 'S2_4076A', 'O1_6300A', 'O1_6364A' , 'O1_6392A',
             'O1_5577A', 'H1_4341A', 'H1_4861A', 'H1_6563A', 'H1_4102A', 'H1_3970A', 'H1_3889A', 'H1_3835A',
             'H1_3798A', 'He1_4471A', 'He1_5016A', 'He1_5876A', 'He1_6678A', 'He1_7065A']
for line in obs.getSortedLines():
    if line.label in ab_labels:
        ab = all_atoms[line.atom].getIonAbundance(line.corrIntens, Te, Ne, to_eval=line.to_eval, Hbeta=100)
        ab_dict[line.atom] = ab


### Include the diagnostics of interest
# instantiate the Diagnostics class
diags = pn.Diagnostics()
# include in diags the relevant line ratios
diags.addDiag([
'[NII] 5755/6584',
'[OII] 3726/3729',
'[OIII] 4363/5007',
'[SII] 6731/6716',
'[ArIV] 4740/4711',
'[SII] 4069/4076',
'[SII] 4072+/6720+',
'[OII] 3727+/7325+',
'[OI] 5577/6300',
'[OI] 5577/6300+',
'[NeIII] 3343/3930+',
'[ArIII] 5192/7136',
'[ArIII] 5192/7300+'  ])

diags.addClabel('[SII] 6731/6716', '[SII]a')
diags.addClabel('[OII] 3726/3729', '[OII]a')
diags.addClabel('[OII] 3727+/7325+', '[OII]b')
diags.addClabel('[SII] 4072+/6720+', '[SII]b')
diags.addClabel('[SII] 4069/4076', '[SII]c')
diags.addClabel('[OI] 5577/6300', '[OI]a')
diags.addClabel('[OI] 5577/6300+', '[OI]b')
diags.addClabel('[ArIII] 5192/7136', '[ArIII]a')
diags.addClabel('[ArIII] 5192/7300+', '[ArIII]b')

# Create the emission maps to be compared to the observation data (some overkill here)
emisgrids = pn.getEmisGridDict(atom_list=diags.getUniqueAtoms(), den_max=1e6)
### Plot
# Create the contour plot as the intersection of tem-den emission maps with dereddened line ra
diags.plot(emisgrids, obs, i_obs=0)
# Place the title
plt.title(title)
plt.savefig(dirName+'/resultTables/diagDenTem'+str(resultListDT['ObjID'][jj])+'D'+str(corrHxHb)+'.png')
print ab_dict

Using matplotlib backend: Qt4Agg
warng _ManageAtomicData: atom data not available for H1
warng _ManageAtomicData: coll data not available for H1
warng _ManageAtomicData: atom data not available for He1
warng _ManageAtomicData: coll data not available for He1
{'S2': array([ nan]), 'N2': array([ nan]), 'O3': array([ nan]), 'O2': array([ nan]), 'O1': array([ nan])}


###                     Plots the fitting spectrum normalized of index jj

In [49]:
#   plot of the galaxy with index jj observed with respect to the continuum in zero. Two models for the lines required 

jj=0

%matplotlib
if isFits:
    if TypeFITS == 0:
        print HDUs[jj][0].header['FLAVOR']
        print HDUs[jj][0].header['QUALITY']
        print HDUs[jj][0].header['FBADPIX'] ## %%Malos Pixeles
        print HDUs[jj][2].data['SN_MEDIAN_ALL']
        print HDUs[jj][2].data['SN_MEDIAN']

lambd=Lambda[jj]

plt.figure(figsize=(16, 9))
plt.title('Spectrum Galaxy '+ resultListDT['ObjID'][jj])
plt.xlabel(r'Wavelength (Angstroms $\AA$) ')
if TypeFITS == 0:
    plt.ylabel(r'$f_{\lambda}$ ('+str(head['BUNIT'])+')')
else:
    plt.ylabel(r'$f_{\lambda}$')

plt.errorbar(Lambda[jj],Spectra[jj]-SpectraCont[jj],SpectraIvar[jj],  c='gray',lw=1)
plt.plot(Lambda[jj], Spectra[jj]-SpectraCont[jj], c='k', lw=0.8, label='Spectrum')
plt.plot(Lambda[jj], Modelos[jj][0](Lambda[jj]), c='yellow', lw=1 , label='Model')

plt.legend()

x=np.linspace(Lambda[jj][0],Lambda[jj][-1],len(Lambda[jj]))
for i in range(len(Tablas[jj])):
    plt.axvline(x=Tablas[jj]['Lambda Vac'][i],c='dimgrey',lw=1.2,ls='--')
    plt.text(Tablas[jj]['Lambda Vac'][i] + 0.5, Tablas[jj]['Amplitude'][i] * 1.2,Tablas[jj]['Element'][i])
    if Tablas[jj]['Element'][i] == 'H I':
        plt.plot(x,gaussian(x,Tablas[jj][i]['Amplitude'],Tablas[jj][i]['Wavelength'],Tablas[jj][i]['Sigma']), c='DeepPink')
##        plt.plot(x,lorentzFunc(x,Tablas[jj][i]['Amplitude'],Tablas[jj][i]['Sigma'], Tablas[jj][i]['Wavelength']), c='DeepPink')
    else:
        if Tablas[jj][i]['Sigma']!=0:
            plt.plot(x,gaussian(x,Tablas[jj][i]['Amplitude'],Tablas[jj][i]['Wavelength'],Tablas[jj][i]['Sigma']) , c='Lime' , label='Modelos')
##            plt.plot(x,lorentzFunc(x,Tablas[jj][i]['Amplitude'],Tablas[jj][i]['Sigma'],Tablas[jj][i]['Wavelength']) , c='Lime')

#plt.savefig('F1.eps', format='eps', dpi=1000)


Using matplotlib backend: Qt4Agg


In [95]:
#TablePyNeb.add_row()
for appa in TablePyNeb['LINE']:
    print appa

O2_3726A
O2_3729A
H1r_4341A
O3_4363A
H1r_4861A
O3_4931A
O3_4959A
O3_5007A
N2_5755A
N2_6527A
N2_6548A
H1r_6563A
N2_6584A
S2_6716A
S2_6731A
Ar4_4711A
Ar4_4740A
H1r_4102A
Fe3_4987A
O2_7319A+
O2_7330A+
S2_4069A
S2_4076A
O1_6300A
O1_6364A
O1_6392A
O1_5577A
H1r_3970A
H1r_3889A
H1r_3835A
H1r_3798A
Ne3_3869A
Ne3_3343A
Ne3_3968A
He1r_4471A
He1r_5016A
He1r_5876A
He1r_6678A
He1r_7065A
Ar3_5192A
Ar3_7751A
Ar3_7136A


In [ ]:
"SDSS J165712.75+321141.4SP1" 254.30315 32.19484 0.03834809 1
3c234SP1 150.45634 28.78582 0.1848955 1


In [119]:
np.mean([resultListDT['Temp_4 [OIII]'][0], resultListDT['Temp_5 [OIII]'][0] ])
np.mean([resultListDT['Den_4 [OII]a'][0], resultListDT['Den_10 [OII]a'][0] ])

230.2844051239245

In [105]:
ab = all_atoms[line.atom].getIonAbundance(line.corrIntens, Te, Ne, to_eval=line.to_eval, Hbeta=100)

In [106]:
ab

array([  1.44719874e-05])

In [120]:
 obs.getSortedLines()

[Line Ar3 Ar3_5192A,
 Line Ar3 Ar3_7136A,
 Line Ar3 Ar3_7751A,
 Line Ar4 Ar4_4711A,
 Line Ar4 Ar4_4740A,
 Line Fe3 Fe3_4987A,
 Line H1r H1r_3798A,
 Line H1r H1r_3835A,
 Line H1r H1r_3889A,
 Line H1r H1r_3970A,
 Line H1r H1r_4102A,
 Line H1r H1r_4341A,
 Line H1r H1r_4861A,
 Line H1r H1r_6563A,
 Line He1r He1r_4471A,
 Line He1r He1r_5016A,
 Line He1r He1r_5876A,
 Line He1r He1r_6678A,
 Line He1r He1r_7065A,
 Line N2 N2_5755A,
 Line N2 N2_6527A,
 Line N2 N2_6548A,
 Line N2 N2_6584A,
 Line Ne3 Ne3_3343A,
 Line Ne3 Ne3_3869A,
 Line Ne3 Ne3_3968A,
 Line O1 O1_5577A,
 Line O1 O1_6300A,
 Line O1 O1_6364A,
 Line O1 O1_6392A,
 Line O2 O2_3726A,
 Line O2 O2_3729A,
 Line O2 O2_7319A+,
 Line O2 O2_7330A+,
 Line O3 O3_4363A,
 Line O3 O3_4931A,
 Line O3 O3_4959A,
 Line O3 O3_5007A,
 Line S2 S2_4069A,
 Line S2 S2_4076A,
 Line S2 S2_6716A,
 Line S2 S2_6731A]

In [121]:
obs.getUniqueAtoms()

array(['Ar3', 'Ar4', 'Fe3', 'H1r', 'He1r', 'N2', 'Ne3', 'O1', 'O2', 'O3',
       'S2'], 
      dtype='|S4')

In [ ]:
obs